In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *

import inspect

## The basics

In [ ]:
#| export
daisy_link = Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css')
tw_scr = Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')
daisy_hdrs = (daisy_link, tw_scr)

In [ ]:
#| export
def mk_previewer(app=None, cls=f'max-w-lg'):
    xcls = cls
    if not app: app=FastHTML(hdrs=daisy_hdrs)
    def p(*c, cls='', **kw):
        return HTMX(Div(cls=f'{xcls} {cls}')(*c), app=app, host=None, port=None, **kw)
    return p

In [ ]:
p = mk_previewer()

In [ ]:
c = Button('Hey there', cls='btn')
p(c)

In [ ]:
print(c)

<button class="btn">Hey there</button>


## Creating simple components

In [ ]:
#| export
def hyphens2camel(x): return ''.join(o.title() for o in x.split('-'))
hyphens2camel('chat-bubble')

'ChatBubble'

In [ ]:
#| export
def mk_compfn(compcls, tag=None, name=None, xcls='', **compkw):
    if not name: name=hyphens2camel(compcls)
    if not tag: tag=name
    compfunc = getattr(fh, tag)

    def fn(*c, cls='', **kw):
        cls = ' '.join(f'{compcls if x[0]=="-" else ""}{x}' for x in cls.split())
        return compfunc(*c, cls=f'{compcls} {cls} {xcls}', **compkw, **kw)

    fn.__name__ = name
    inspect.currentframe().f_back.f_globals[name] = fn

In [ ]:
mk_compfn('btn', 'Button')

In [ ]:
c = Btn('Hey there', cls='-primary p-5 text-2xl rounded-full')
print(c)

<button class="btn btn-primary p-5 text-2xl rounded-full ">Hey there</button>


In [ ]:
p(c)